In [1]:
import psycopg2
import pymongo
import numpy as np
import pandas as pd

conn = psycopg2.connect("dbname=mydb user=postgres")
cur = conn.cursor()

connection = pymongo.MongoClient()
db = connection.test
following = db.following

# Import MongoDB and PostgreSQL API and connect to server

In [2]:
cur.execute("DELETE FROM following")
conn.commit()
# Clear psql table named following

following.delete_many({})
# Clear mongodb collection named following

In [3]:
for x in xrange(500):
    cur.execute("INSERT INTO following (followee_id, follower_id) VALUES (%s, %s)", 
                (np.random.randint(1,101), np.random.randint(1,101)))
conn.commit()
# Insert random interger 1 ~ 100 as followee's id and follower's id in psql

for x in xrange(500):
    following.insert_one({"followee_id": np.random.randint(1,101), 
                          "follower_id": np.random.randint(1,101)})
# Insert random interger 1 ~ 100 as followee's id and follower's id in mongodb

In [4]:
cur.execute("SELECT * FROM following LIMIt 6;")
cur.fetchall()
# Have a look at the data in psql

[(12, 46), (56, 17), (88, 16), (14, 55), (93, 34), (49, 66)]

In [5]:
a = following.find().limit(6)
for x in a:
    print x
# Have a look at the data in mongodb

{u'follower_id': 55, u'_id': ObjectId('55b9320a44204945d4f19176'), u'followee_id': 78}
{u'follower_id': 9, u'_id': ObjectId('55b9320a44204945d4f19177'), u'followee_id': 37}
{u'follower_id': 51, u'_id': ObjectId('55b9320a44204945d4f19178'), u'followee_id': 49}
{u'follower_id': 57, u'_id': ObjectId('55b9320a44204945d4f19179'), u'followee_id': 93}
{u'follower_id': 59, u'_id': ObjectId('55b9320a44204945d4f1917a'), u'followee_id': 87}
{u'follower_id': 80, u'_id': ObjectId('55b9320a44204945d4f1917b'), u'followee_id': 4}


In [6]:
followee_id = range(1,101)
follower_2nd_degree_psql = []
for x in xrange(1,101):
    cur.execute("""SELECT DISTINCT follower_id FROM following WHERE followee_id in (
                   SELECT DISTINCT follower_id FROM following WHERE followee_id = %d)""" %x)
    follower_2nd_degree_psql.append(len(cur.fetchall()))
# Query the 2nd degree follower for each followee and count the number in psql

In [7]:
follower_2nd_degree_mongo = []
for x in xrange(1,101):
    follower_1st = following.distinct("follower_id", {"followee_id": x})
    follower_2nd = following.distinct("follower_id", {"followee_id": {"$in": follower_1st}})
    follower_2nd_degree_mongo.append(len(follower_2nd))
# Query the 2nd degree follower for each followee and count the number in mongodb

In [8]:
d_psql = {"followee_id" : followee_id, 
          "follower_2nd_degree_count" : follower_2nd_degree_psql}
pd.DataFrame(d_psql).head(10)
# Show the number of 2nd degree follower for each followee in pandas dataframe, using psql

,followee_id,follower_2nd_degree_count
0,1,5
1,2,24
2,3,18
3,4,28
4,5,28
5,6,11
6,7,21
7,8,11
8,9,10
9,10,9


In [9]:
d_mongo = {"followee_id" : followee_id, 
           "follower_2nd_degree_count" : follower_2nd_degree_mongo}
pd.DataFrame(d_mongo).head(10)
# Show the number of 2nd degree follower for each followee in pandas dataframe, using mongodb

,followee_id,follower_2nd_degree_count
0,1,21
1,2,7
2,3,23
3,4,40
4,5,19
5,6,27
6,7,11
7,8,23
8,9,15
9,10,23


In [ ]:
# Notes: 1. Someone in the list of 2nd degree follower are also a 1st degree follower, I didn't 
#           exclude them. 
#        2. The result from postgresql and mongodb shown in pandas dataframe are not the same 
#           because I insert data into their servers respectively with random integers. 